In [2]:
import glob
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datasets import load_dataset




In [ ]:
# Load LLaMA Model

model_id = "meta-llama/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [8]:
# Load Documents from Path using TextLoader

ds = load_dataset("gwenshap/sales-transcripts")

Generating train split: 100%|██████████| 1935/1935 [00:00<00:00, 10772.27 examples/s]


In [9]:
# Print the first few examples
print(ds['train'][:5])


{'text': ['**Sales Rep**: Hi there! Thank you for taking the time to speak with me today. My name is Jamie, and I’m a sales representative from ModaMart. How are you today?', '', "**Customer**: Hi Jamie. I'm doing well, thanks. How about you?", '', '**Sales Rep**: I’m great, thank you! So, I understand you’ve been browsing our online store. Is there anything specific you’re looking for today?']}


In [ ]:
# Split large documents into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(ds)

In [ ]:
# Embed the documents and initialize Chroma vector store

embedding_model = HuggingFaceEmbeddings(model_id="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma(embedding_function=embedding_model, persist_directory="./db")
vector_store.add_documents(split_documents)
vector_store.persist()

In [ ]:
# Create a Retriever and QA Chain

retriever = vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=model, retriever=retriever)

In [ ]:
# Set Up a Simple Query Function to Run RAG

def ask_query(query):
    response = qa_chain.run(query)
    return response

In [ ]:
# Query the Model

query = "What is Hugging Face."
response = ask_query(query)

In [ ]:
print(response)